In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
import time

from multiprocess import Pool

sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *
from metadata import *
from data_manager import *
from visualization_utilities import *

Setting environment for AWS compute node


No vtk
File does not exist: /shared/CSHL_data_processed/MD635/MD635_anchor.txt
File does not exist: /shared/CSHL_data_processed/MD635/MD635_sorted_filenames.txt
File does not exist: /shared/CSHL_data_processed/MD635/MD635_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD635/MD635_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD653/MD653_anchor.txt
File does not exist: /shared/CSHL_data_processed/MD653/MD653_sorted_filenames.txt
File does not exist: /shared/CSHL_data_processed/MD653/MD653_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD653/MD653_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD652/MD652_anchor.txt
File does not exist: /shared/CSHL_data_processed/MD652/MD652_sorted_filenames.txt
File does not exist: /shared/CSHL_data_processed/MD652/MD652_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD652/MD652_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD642/MD642_anchor.txt
File does not exist: 

In [2]:
stack = 'MD591'

In [10]:
classifier_id = 30
downscale = 8

# One section

In [15]:
sec = 290
actual_setting = resolve_actual_setting(setting=classifier_id, stack=stack, sec=sec)
# DataManager.get_downscaled_scoremap_viz_filepath(stack=stack, section=sec, structure='7N', setting=actual_setting)
viz = scoremap_overlay(stack=stack, sec=sec, structure='7N', downscale=downscale, setting=actual_setting)
# viz = scoremap_overlay_on(bg=bg, stack=stack, sec=sec, structure='7N', 
#                           downscale=downscale, label_text=None, setting=actual_setting)

In [16]:
display_image(viz, filename='tmp.png')

/shared/MouseBrainAtlas/learning/tmp.png

# Many sections

In [4]:
first_sec, last_sec = metadata_cache['section_limits'][stack]

In [5]:
add_label_text = False

In [13]:
# for sec in range(200, 202):
for sec in [270]:
# for sec in range(first_sec, last_sec+1):
    
    if is_invalid(stack=stack, sec=sec):
        continue

    actual_setting = resolve_actual_setting(setting=classifier_id, stack=stack, sec=sec)

    t = time.time()

    def save_scoremap(structure):
        viz_fp = DataManager.get_scoremap_viz_filepath(stack=stack, downscale=downscale, section=sec, structure=structure, setting=actual_setting)
        create_parent_dir_if_not_exists(viz_fp)
        try:
            if add_label_text:
                label_text = str(structure)
            else:
                label_text = None

            viz = scoremap_overlay_on(bg='original', stack=stack, sec=sec, structure=structure,
                                downscale=downscale, label_text=label_text, setting=actual_setting)
            imsave(viz_fp, img_as_ubyte(viz))
            upload_from_ec2_to_s3(viz_fp)
        except Exception as e:
            sys.stderr.write('%s\n' % e)
            return
    
    pool = Pool(NUM_CORES)
    pool.map(save_scoremap, all_known_structures)
    pool.close()
    pool.join()

    sys.stderr.write('Visualize scoremaps: %.2f seconds.\n' % (time.time() - t)) 
    # 7s for one structure, one section, single process
    # 20s for all structures, one section, 8 processes

/usr/local/lib/python2.7/dist-packages/PIL/Image.py:2371: DecompressionBombWarning: Image size (209920000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)
/usr/local/lib/python2.7/dist-packages/PIL/Image.py:2371: DecompressionBombWarning: Image size (209920000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)
/usr/local/lib/python2.7/dist-packages/PIL/Image.py:2371: DecompressionBombWarning: Image size (209920000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)
/usr/local/lib/python2.7/dist-packages/PIL/Image.py:2371: DecompressionBombWarning: Image size (209920000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)
/usr/local/lib/python2.7/dist-packages/PIL/Image.py:2371: DecompressionBombWarning: Image size (209920000 pixels) exceeds limit of 89478

aws s3 cp /shared/CSHL_scoremaps_down8_viz/SNR/MD591/MD591-IHC58-2015.08.28-07.09.16_MD591_2_0173_lossless_alignedTo_MD591-N57-2015.09.21-17.13.29_MD591_1_0169_cropped_8_SNR_denseScoreMap_setting_30.jpg s3://mousebrainatlas-data/CSHL_scoremaps_down8_viz/SNR/MD591/MD591-IHC58-2015.08.28-07.09.16_MD591_2_0173_lossless_alignedTo_MD591-N57-2015.09.21-17.13.29_MD591_1_0169_cropped_8_SNR_denseScoreMap_setting_30.jpg
aws s3 cp /shared/CSHL_scoremaps_down8_viz/5N/MD591/MD591-IHC58-2015.08.28-07.09.16_MD591_2_0173_lossless_alignedTo_MD591-N57-2015.09.21-17.13.29_MD591_1_0169_cropped_8_5N_denseScoreMap_setting_30.jpg s3://mousebrainatlas-data/CSHL_scoremaps_down8_viz/5N/MD591/MD591-IHC58-2015.08.28-07.09.16_MD591_2_0173_lossless_alignedTo_MD591-N57-2015.09.21-17.13.29_MD591_1_0169_cropped_8_5N_denseScoreMap_setting_30.jpg
aws s3 cp /shared/CSHL_scoremaps_down8_viz/LRt/MD591/MD591-IHC58-2015.08.28-07.09.16_MD591_2_0173_lossless_alignedTo_MD591-N57-2015.09.21-17.13.29_MD591_1_0169_cropped_8_LRt_de

Child returned 0
0.73 seconds.
Child returned 0
Child returned 0
0.77 seconds.
Child returned 0
0.72 seconds.
0.62 seconds.
Child returned 0
0.73 seconds.
Child returned 0
0.71 seconds.
Child returned 0
0.73 seconds.
Child returned 0
0.72 seconds.
Child returned 0
0.62 seconds.
Child returned 0
0.67 seconds.
Child returned 0
0.76 seconds.
Child returned 0
Child returned 0
0.67 seconds.
0.71 seconds.
Child returned 0
0.72 seconds.
Child returned 0
0.79 seconds.
Child returned 0
0.72 seconds.
Error loading scoremap of sp5 for image MD591-IHC58-2015.08.28-07.09.16_MD591_2_0173.
Error loading scoremap of outerContour for image MD591-IHC58-2015.08.28-07.09.16_MD591_2_0173.


aws s3 cp /shared/CSHL_scoremaps_down8_viz/10N/MD591/MD591-IHC58-2015.08.28-07.09.16_MD591_2_0173_lossless_alignedTo_MD591-N57-2015.09.21-17.13.29_MD591_1_0169_cropped_8_10N_denseScoreMap_setting_30.jpg s3://mousebrainatlas-data/CSHL_scoremaps_down8_viz/10N/MD591/MD591-IHC58-2015.08.28-07.09.16_MD591_2_0173_lossless_alignedTo_MD591-N57-2015.09.21-17.13.29_MD591_1_0169_cropped_8_10N_denseScoreMap_setting_30.jpg
aws s3 cp /shared/CSHL_scoremaps_down8_viz/DC/MD591/MD591-IHC58-2015.08.28-07.09.16_MD591_2_0173_lossless_alignedTo_MD591-N57-2015.09.21-17.13.29_MD591_1_0169_cropped_8_DC_denseScoreMap_setting_30.jpg s3://mousebrainatlas-data/CSHL_scoremaps_down8_viz/DC/MD591/MD591-IHC58-2015.08.28-07.09.16_MD591_2_0173_lossless_alignedTo_MD591-N57-2015.09.21-17.13.29_MD591_1_0169_cropped_8_DC_denseScoreMap_setting_30.jpg
aws s3 cp /shared/CSHL_scoremaps_down8_viz/VCP/MD591/MD591-IHC58-2015.08.28-07.09.16_MD591_2_0173_lossless_alignedTo_MD591-N57-2015.09.21-17.13.29_MD591_1_0169_cropped_8_VCP_de

Child returned 0
0.81 seconds.


...

Child returned 0
0.70 seconds.


aws s3 cp /shared/CSHL_scoremaps_down8_viz/RtTg/MD591/MD591-IHC58-2015.08.28-07.09.16_MD591_2_0173_lossless_alignedTo_MD591-N57-2015.09.21-17.13.29_MD591_1_0169_cropped_8_RtTg_denseScoreMap_setting_30.jpg s3://mousebrainatlas-data/CSHL_scoremaps_down8_viz/RtTg/MD591/MD591-IHC58-2015.08.28-07.09.16_MD591_2_0173_lossless_alignedTo_MD591-N57-2015.09.21-17.13.29_MD591_1_0169_cropped_8_RtTg_denseScoreMap_setting_30.jpg
aws s3 cp /shared/CSHL_scoremaps_down8_viz/AP/MD591/MD591-IHC58-2015.08.28-07.09.16_MD591_2_0173_lossless_alignedTo_MD591-N57-2015.09.21-17.13.29_MD591_1_0169_cropped_8_AP_denseScoreMap_setting_30.jpg s3://mousebrainatlas-data/CSHL_scoremaps_down8_viz/AP/MD591/MD591-IHC58-2015.08.28-07.09.16_MD591_2_0173_lossless_alignedTo_MD591-N57-2015.09.21-17.13.29_MD591_1_0169_cropped_8_AP_denseScoreMap_setting_30.jpg
aws s3 cp /shared/CSHL_scoremaps_down8_viz/12N/MD591/MD591-IHC58-2015.08.28-07.09.16_MD591_2_0173_lossless_alignedTo_MD591-N57-2015.09.21-17.13.29_MD591_1_0169_cropped_8_12

Child returned 0
0.70 seconds.
Child returned 0
0.61 seconds.
Child returned 0
0.60 seconds.


aws s3 cp /shared/CSHL_scoremaps_down8_viz/VCA/MD591/MD591-IHC58-2015.08.28-07.09.16_MD591_2_0173_lossless_alignedTo_MD591-N57-2015.09.21-17.13.29_MD591_1_0169_cropped_8_VCA_denseScoreMap_setting_30.jpg s3://mousebrainatlas-data/CSHL_scoremaps_down8_viz/VCA/MD591/MD591-IHC58-2015.08.28-07.09.16_MD591_2_0173_lossless_alignedTo_MD591-N57-2015.09.21-17.13.29_MD591_1_0169_cropped_8_VCA_denseScoreMap_setting_30.jpg
aws s3 cp /shared/CSHL_scoremaps_down8_viz/IC/MD591/MD591-IHC58-2015.08.28-07.09.16_MD591_2_0173_lossless_alignedTo_MD591-N57-2015.09.21-17.13.29_MD591_1_0169_cropped_8_IC_denseScoreMap_setting_30.jpg s3://mousebrainatlas-data/CSHL_scoremaps_down8_viz/IC/MD591/MD591-IHC58-2015.08.28-07.09.16_MD591_2_0173_lossless_alignedTo_MD591-N57-2015.09.21-17.13.29_MD591_1_0169_cropped_8_IC_denseScoreMap_setting_30.jpg
aws s3 cp /shared/CSHL_scoremaps_down8_viz/SC/MD591/MD591-IHC58-2015.08.28-07.09.16_MD591_2_0173_lossless_alignedTo_MD591-N57-2015.09.21-17.13.29_MD591_1_0169_cropped_8_SC_dens

Child returned 0
0.46 seconds.
Child returned 0
0.62 seconds.
Child returned 0
0.45 seconds.
Child returned 0
0.55 seconds.
Child returned 0
0.46 seconds.
Child returned 0
0.50 seconds.
Child returned 0
0.44 seconds.
Visualize scoremaps: 22.40 seconds.
